# Project: Recognizing Traffic Signs Using Deep Learning
### Author: Ashish Panchal (epababl03.ashishp@iima.ac.in)

In [12]:
# Load/Import Libraries
import sys
import os
import numpy as np
import statistics
import matplotlib.pyplot as plt

import keras

from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.layers import Dropout 
from keras.layers.normalization import BatchNormalization
from keras import regularizers  
from keras.optimizers import SGD
from keras.layers import Flatten, Conv2D, MaxPooling2D 
from keras.callbacks import ModelCheckpoint

In [13]:
#Load stored data

# No. of classes
%store -r n_classes

# class labels
%store -r signs

# Training Dataset
%store -r X_train
%store -r y_train

# Test Dataset
%store -r X_test
%store -r y_test


# Validation Dataset
%store -r X_valid
%store -r y_valid


# 3. Data Preprocessing 

### 3.1 Preprocessing Techniques
We have applied several preprocessing steps to the input images to achieve the best possible results. we have used the following preprocessing techniques:
1. Data Augmentation
    - Slight Rotation of Images
    - Image Translation
2. Shuffling
3. Bilateral Filtering
    - Grayscaling
    - Local Histogram Equalization
    - Normalization


Before we do any data preprocessing, lets have simple dense network to baseline quality of data on basis model accuracy.
#### Model Testing without any preprocessing - Establishing Baseline - Neural Network Architecture

In [14]:
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(32*32*3,)))
model.add(BatchNormalization())
model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(n_classes, activation='softmax'))

In [15]:
model.summary()
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_11 (Dense)             (None, 128)               393344    
_________________________________________________________________
batch_normalization_9 (Batch (None, 128)               512       
_________________________________________________________________
dense_12 (Dense)             (None, 128)               16512     
_________________________________________________________________
batch_normalization_10 (Batc (None, 128)               512       
_________________________________________________________________
dropout_5 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 128)               16512     
_________________________________________________________________
batch_normalization_11 (Batc (None, 128)              

In [16]:
X_train_baseline = X_train.reshape(len(X_train), 32*32*3).astype('float32')
X_valid_baseline = X_valid.reshape(len(X_valid), 32*32*3).astype('float32')
y_train_baseline = keras.utils.to_categorical(y_train, n_classes)
y_valid_baseline = keras.utils.to_categorical(y_valid, n_classes)

In [ ]:
base_history = model.fit(X_train_baseline, y_train_baseline, batch_size=128, 
                         epochs=100, verbose=1, validation_data=(X_valid_baseline, y_valid_baseline))


Train on 34799 samples, validate on 4410 samples
Epoch 1/100
34799/34799 [==============================] - 9s 250us/step - loss: 2.6912 - accuracy: 0.2776 - val_loss: 2.1739 - val_accuracy: 0.3422
Epoch 2/100
34799/34799 [==============================] - 6s 166us/step - loss: 1.3719 - accuracy: 0.5757 - val_loss: 1.6383 - val_accuracy: 0.5152
Epoch 3/100
34799/34799 [==============================] - 6s 175us/step - loss: 0.9747 - accuracy: 0.7006 - val_loss: 1.6049 - val_accuracy: 0.5821
Epoch 4/100
34799/34799 [==============================] - 6s 173us/step - loss: 0.7517 - accuracy: 0.7685 - val_loss: 1.1461 - val_accuracy: 0.6871
Epoch 5/100
34799/34799 [==============================] - 6s 174us/step - loss: 0.6658 - accuracy: 0.7969 - val_loss: 1.0965 - val_accuracy: 0.7181
Epoch 6/100
34799/34799 [==============================] - 6s 174us/step - loss: 0.5892 - accuracy: 0.8214 - val_loss: 1.2484 - val_accuracy: 0.6685
Epoch 7/100
34799/34799 [==============================]